# Are They Siblings or Dating?
## A Deep Learning Image Classification Project
### By Andy Phan, Sanjay Karunamoorthy, Kevin Arleen

In [2]:
import praw
import requests
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
from tensorflow.keras.preprocessing.image import load_img, img_to_array
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
reddit = praw.Reddit(
    client_id="SZ6ae6M47WrdQ37nACw10Q",
    client_secret="XqzJnY1D3g2_uy1Mlu-Su65lsa2fAg",
    user_agent="SiblingsOrDatingScraper"
)

In [4]:
print(reddit.read_only)

True


In [5]:
subreddit = reddit.subreddit("siblingsordating")
for submission in subreddit.hot(limit=10):
    print(submission.title)

We fixed the No Answer issue...hopefully. PLEASE READ POSTERS
Siblings or dating
SOD?
Siblings or Dating?
Siblings or Dating?
siblings or dating?
Siblings or dating?
Siblings or dating
Siblings or dating?
SOD


In [6]:
base_dir = "images"
os.makedirs(os.path.join(base_dir, "siblings"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "dating"), exist_ok=True)

for i, post in enumerate(subreddit.top(limit=1000)):
    if post.url.endswith(('.jpg', '.png', '.jpeg', '.webp')):
        post.comments.replace_more(limit=0)
        author_label = None

        def extract_label(text):
            text = text.lower()
            if 'sib' in text and 'dating' not in text:
                return 'siblings'
            elif ('dating' in text or 'couple' in text) and 'sib' not in text:
                return 'dating'
            return None

        # Step 1: Check top-level comments from the author
        for comment in post.comments:
            if comment.author and post.author and comment.author.name == post.author.name:
                author_label = extract_label(comment.body)
                if author_label:
                    break

        # Step 2: Check replies to others if no label found yet
        if not author_label:
            for comment in post.comments:
                for reply in comment.replies:
                    if reply.author and post.author and reply.author.name == post.author.name:
                        author_label = extract_label(reply.body)
                        if author_label:
                            break
                if author_label:
                    break

        # Save image to correct subfolder
        if author_label:
            try:
                label_dir = os.path.join(base_dir, author_label)
                filename = os.path.join(label_dir, f"{i}.jpg")
                image_data = requests.get(post.url).content
                with open(filename, 'wb') as f:
                    f.write(image_data)
                print(f"Saved: {filename} | Label: {author_label}")
            except Exception as e:
                print(f"Failed to download image: {e}")
        else:
            print(f"Skipped post {i}: No author label found")

Skipped post 0: No author label found
Saved: images\dating\1.jpg | Label: dating
Skipped post 2: No author label found
Saved: images\siblings\3.jpg | Label: siblings
Saved: images\dating\4.jpg | Label: dating
Skipped post 5: No author label found
Saved: images\dating\6.jpg | Label: dating
Saved: images\dating\7.jpg | Label: dating
Skipped post 8: No author label found
Saved: images\siblings\9.jpg | Label: siblings
Skipped post 12: No author label found
Saved: images\dating\13.jpg | Label: dating
Saved: images\siblings\15.jpg | Label: siblings
Saved: images\dating\17.jpg | Label: dating
Skipped post 18: No author label found
Saved: images\dating\19.jpg | Label: dating
Saved: images\siblings\20.jpg | Label: siblings
Saved: images\dating\21.jpg | Label: dating
Saved: images\siblings\22.jpg | Label: siblings
Saved: images\dating\23.jpg | Label: dating
Saved: images\dating\24.jpg | Label: dating
Saved: images\dating\25.jpg | Label: dating
Saved: images\dating\26.jpg | Label: dating
Saved: i

In [7]:
data_dir = "/content/images"
from google.colab import files
import zipfile

uploaded = files.upload()
with zipfile.ZipFile("images.zip", "r") as zip_ref:
  zip_ref.extractall("images")
empty_files = []

for root, _, files in os.walk(data_dir):
    for file in files:
        path = os.path.join(root, file)
        if os.path.getsize(path) == 0:
            empty_files.append(path)

if empty_files:
    print("Empty image files found:")
    for path in empty_files:
        print(path)
else:
    print("No empty files found.")
for path in empty_files:
    os.remove(path)
    print(f"Deleted: {path}")

ModuleNotFoundError: No module named 'google.colab'

In [8]:
valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
removed = 0

for root, _, files in os.walk("images"):
    for file in files:
        path = os.path.join(root, file)
        ext = os.path.splitext(file)[1].lower()
        if ext not in valid_exts or os.path.getsize(path) == 0:
            os.remove(path)
            print(f"Deleted: {path}")
            removed += 1

print(f"Cleanup complete. Removed {removed} invalid or empty files.")

Deleted: images\dating\177.jpg
Deleted: images\siblings\126.jpg
Deleted: images\siblings\151.jpg
Cleanup complete. Removed 3 invalid or empty files.


In [9]:
from PIL import Image

for root, _, files in os.walk("images"):
    for file in files:
        if file.endswith(".webp"):
            path = os.path.join(root, file)
            try:
                img = Image.open(path).convert("RGB")
                new_path = path.replace(".webp", ".jpg")
                img.save(new_path, "JPEG")
                os.remove(path)
                print(f"Converted: {file} → .jpg")
            except Exception as e:
                print(f"Failed to convert {file}: {e}")

In [10]:
bad_files = []

for root, _, files in os.walk("images"):
    for file in files:
        filepath = os.path.join(root, file)
        try:
            img = tf.io.read_file(filepath)
            img = tf.image.decode_image(img, channels=3)
        except Exception as e:
            print(f"Corrupted or unreadable file: {filepath} | Error: {e}")
            bad_files.append(filepath)

print(f"\nFound {len(bad_files)} bad files.")

for path in bad_files:
    os.remove(path)
    print(f"Deleted: {path}")


Found 0 bad files.


In [11]:
import shutil
from google.colab import files

shutil.make_archive("images", "zip", "images")
files.download("images.zip")

ModuleNotFoundError: No module named 'google.colab'

In [12]:
batch_size = 32
img_height, img_width = 224, 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: '/content/images'

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

rescale_layer = Rescaling(1/.255)

data_augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
])

def preprocess_images(images, labels):
    images = rescale_layer(images)
    images = data_augmentation(images)
    return images, labels

train_ds = train_ds.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)

# For validation, just rescale (no augmentation):
validation_ds = validation_ds.map(lambda x, y: (rescale_layer(x), y),
                                  num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.shuffle(1000).prefetch(tf.data.AUTOTUNE)
validation_ds = validation_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
class_names = ['Siblings', 'Dating']

n_rows = 8
n_cols = 4
plt.figure(figsize=(n_cols * 3, n_rows * 3))
for images, labels in train_ds.take(1):
  for i in range(n_rows*n_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis('off')
    plt.title(class_names[labels[i]], fontsize=12)
  plt.subplots_adjust(wspace=.2, hspace=.2)

In [ ]:
from tensorflow.keras import layers, models

model = keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_cb = ModelCheckpoint(
    "best_model.keras",
    save_best_only=True,
    monitor="val_accuracy",
    mode="auto"
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    verbose=1
    )

y_train = np.concatenate([y.numpy() for _, y in train_ds], axis=0)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

base_model = MobileNetV2(input_shape=(224, 224, 3),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=5,
    callbacks=[early_stop, reduce_lr, checkpoint_cb],
    class_weight=class_weights_dict
)

myEpochs = 30
myOptimizer = Adam(learning_rate=0.0001)
myLoss = 'binary_crossentropy'
myMetrics = ['accuracy']
model.compile(
    loss=myLoss,
    optimizer=myOptimizer,
    metrics=myMetrics
)
history = model.fit(train_ds,
                    validation_data=validation_ds,
                    epochs=myEpochs,
                    callbacks=[early_stop, reduce_lr],
                    class_weight=class_weights_dict)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 794ms/step - accuracy: 0.5893 - loss: 1.0829 - val_accuracy: 0.7368 - val_loss: 0.5713 - learning_rate: 1.0000e-04
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 313ms/step - accuracy: 0.6071 - loss: 0.8172 - val_accuracy: 0.7368 - val_loss: 0.5772 - learning_rate: 1.0000e-04
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 317ms/step - accuracy: 0.6388 - loss: 0.8502 - val_accuracy: 0.7105 - val_loss: 0.5874 - learning_rate: 1.0000e-04
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 397ms/step - accuracy: 0.5244 - loss: 0.8337 - val_accuracy: 0.7632 - val_loss: 0.5978 - learning_rate: 1.0000e-04
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 319ms/step - accuracy: 0.5579 - loss: 0.8289 - val_accuracy: 0.6842 - val_loss: 0.6105 - learning_rate: 1.0000e-04


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

model.compile(optimizer=Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=100,
    callbacks=[early_stop, reduce_lr, checkpoint_cb],
    class_weight=class_weights_dict
)

best_model = tf.keras.models.load_model("best_model.keras")

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 734ms/step - accuracy: 0.5345 - loss: 0.7342 - val_accuracy: 0.7105 - val_loss: 0.5755 - learning_rate: 1.0000e-05
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 363ms/step - accuracy: 0.5738 - loss: 0.7169 - val_accuracy: 0.7368 - val_loss: 0.5790 - learning_rate: 1.0000e-05
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 348ms/step - accuracy: 0.5104 - loss: 0.7393 - val_accuracy: 0.7368 - val_loss: 0.5820 - learning_rate: 1.0000e-05
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 350ms/step - accuracy: 0.5680 - loss: 0.6899 - val_accuracy: 0.7105 - val_loss: 0.5843 - learning_rate: 1.0000e-05
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.5405 - loss: 0.7041
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 353ms/step - accuracy: 0.5446 - loss: 0.7027 - val_accuracy: 0.7105 - val_loss: 0.5863 - learning_rate: 1.0000e-05
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 350ms/step - accuracy: 0.5967 - los

In [ ]:
model.evaluate(validation_ds)
best_model.evaluate(validation_ds)

In [ ]:
!rm -rf images
best_val_loss, best_val_acc = best_model.evaluate(validation_ds)
print("Best saved model - val_loss:", best_val_loss, "val_accuracy:", best_val_acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.7692 - loss: 0.5930
Best saved model - val_loss: 0.5977979898452759 val_accuracy: 0.7631579041481018
